In [1]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
def fixPhi(df, particle, groomed=False):
    if groomed == True:
        string = 'phi_jet_g'
    else:
        string = 'phi_jet'
    a = (df.loc[df['phi_' + particle] < 0]).index # a list of row indices (jet id) with negative phi_photon values
    for i in range(0,len(a)):
        df.at[a[i],'phi_' + particle] = df.at[a[i],'phi_' + particle] + 6.28318530718
    print 'Fixed ' + particle + ' phi range'
    df_new = df.eval('delta_phi_' + particle + ' = phi_' + particle + ' - ' + string)
    b = (df_new.loc[df_new['delta_phi_' + particle] > np.pi]).index
    for i in range(0,len(b)):
        df_new.at[b[i],'delta_phi_' + particle] = -6.28318530718 + df_new.at[b[i],'delta_phi_' + particle]
    c = (df_new.loc[df_new['delta_phi_' + particle] < (-1) * np.pi]).index
    for i in range(0,len(c)):
        df_new.at[c[i],'delta_phi_' + particle] = 6.28318530718 + df_new.at[c[i],'delta_phi_' + particle]
    df_new_new = df_new.eval('delta_phi_' + particle + '_abs = abs(delta_phi_' + particle + ')')    
    print 'Fixed ' + particle + ' delta phi range'
    return df_new_new

In [1]:
def fixPhiPart(df, particle):
    string = 'phi_part'
    a = (df.loc[df['phi_' + particle] < 0]).index # a list of row indices (jet id) with negative phi_photon values
    for i in range(0,len(a)):
        df.at[a[i],'phi_' + particle] = df.at[a[i],'phi_' + particle] + 6.28318530718
    print 'Fixed ' + particle + ' phi range'
    df_new = df.eval('delta_phi_' + particle + ' = phi_' + particle + ' - ' + string)
    b = (df_new.loc[df_new['delta_phi_' + particle] > np.pi]).index
    for i in range(0,len(b)):
        df_new.at[b[i],'delta_phi_' + particle] = -6.28318530718 + df_new.at[b[i],'delta_phi_' + particle]
    c = (df_new.loc[df_new['delta_phi_' + particle] < (-1) * np.pi]).index
    for i in range(0,len(c)):
        df_new.at[c[i],'delta_phi_' + particle] = 6.28318530718 + df_new.at[c[i],'delta_phi_' + particle]
    df_new_new = df_new.eval('delta_phi_' + particle + '_abs = abs(delta_phi_' + particle + ')')    
    print 'Fixed ' + particle + ' delta phi range'
    return df_new_new

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [1]:
def plot1dnvar(df_array, var_array, label_array, low, high, interval, xlabel, yscale):
    
    color_array = ['r', 'g', 'b', 'purple']
    xvar_array = toArrayFloat(low, high, interval)
    xsec = np.mean(a['xsec'])
    ntrials = a['ntrials'][a.shape[0]-1]
    
    for i in range(0, len(df_array)):
        x, y, xerr = getHist(df_array[i], var_array[i], xvar_array)
        plt.errorbar(x, 10*y*xsec*1e12/ntrials, xerr = xerr, fmt='none', color=color_array[i],elinewidth=2,label=label_array[i])
   
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.yscale(yscale)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel(r'$d\sigma \times 10 \, \mathrm{fb^{-1}}$', fontsize=20)
    plt.xlim(left=low, right=high)
    plt.yscale(yscale)
    plt.legend(prop={'size': 20}, frameon=False, loc='upper right')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [4]:
def toArrayFloat(begin, end, interval=0, log=False, logNum=0):
    array = []
    if log==True:
        array = np.logspace(begin, end, num=logNum)
    else:
        length = int((end - begin) / interval + 1)
        for i in range(0,length):
            array.append((float(begin + interval * i))) 
    return array

In [1]:
def getHist(var, binEdges):
        
    x = []
    for i in range(0,len(binEdges)-1):
        x.append((binEdges[i] + binEdges[i+1]) / 2)
    
    y = np.histogram(var, bins=binEdges)[0]
    
    xerr = getInterval(binEdges)/2
    
    return x, y, xerr

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [6]:
def getInterval(arrayFloat):
    interval = np.zeros(len(arrayFloat)-1)
    for i in range(0,len(arrayFloat)-1):
        interval[i] = arrayFloat[i+1] - arrayFloat[i]
    return interval

In [1]:
def partitionEtaPt(df, etaedges, ptranges):
    etaranges = getRanges(etaedges)
    ptranges = getRanges(ptranges)
    ds = [['' for j in range(0,len(ptranges))] for i in range(0,len(etaranges))]
    for i in range(0,len(etaranges)):
        etalow, etahigh = etaranges[i]
        print '\n', str(etalow) +' < eta_jet < '+ str(etahigh), '\n'
        ds_eta = applyCut(df, 'eta_jet> ' + str(etalow) + 'and eta_jet < ' + str(etahigh), str(etalow) + ' < eta_jet < ' + str(etahigh))
        for j in range(0,len(ptranges)):
            ptlow, pthigh = ptranges[j]
            ds_eta_pt = applyCut(ds_eta, 'pt_jet > ' + str(ptlow) + 'and pt_jet < ' + str(pthigh), str(ptlow) + ' < pt_jet < ' + str(pthigh))
            ds[i][j] = ds_eta_pt
    return ds

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [11]:
def partitionEta(df, etaedges, etaname='eta_jet'):
    etaranges = getRanges(etaedges)
    ds = ['' for i in range(0,len(etaranges))]
    for i in range(0,len(etaranges)):
        etalow, etahigh = etaranges[i]
        ds_eta = applyCut(df, etaname + ' > ' + str(etalow) + ' and ' + etaname + ' < ' + str(etahigh), str(etalow) + ' < ' + etaname + ' < ' + str(etahigh))
        ds[i] = ds_eta
    return ds

In [9]:
def getRanges(edges):
    return zip(edges[:-1], edges[1:])

In [1]:
def keepOneJet(inputDataframe):    
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.loc[~dataframe['event'].duplicated(keep='first')]
    print '\n','Remove extra jets',cutDataframe.shape[0],' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [3]:
def plot2d(df, xvar, yvar, xlabel, ylabel, title=None, lim=None, cmap='inferno'):

    hist = plt.hexbin(df[xvar], df[yvar], gridsize=50, norm=LogNorm(), cmap=cmap, xscale = 'log', extent=lim)
    plt.xlabel(xlabel, fontsize = 15)
    plt.ylabel(ylabel, fontsize = 15)
    plt.tick_params(labelsize='large')
    plt.colorbar()
    if title != None:
        plt.title('$' + title + '$', fontsize=15)
    return hist

In [1]:
def integrate(binwidth, yarray, low, high):
    s = 0
    interval = (int)((high - low)/binwidth)
    for i in range(0, interval):
        s = s + binwidth * yarray[i]
    return s

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [1]:
def find_dup(df, file_name='default'): # find rows with duplicated "matched" jets
    dup_file = open("dup/" + file_name + ".txt", "w") 
    for i in range(0, len(df)):
        match = np.asarray(df['matched_truthPhi'])[i]
        if i < len(df)-20:
            end = i + 21
        else:
            end = len(df)
        for j in range(i+1, end):
            match2 = np.asarray(df['matched_truthPhi'])[j]
            if match == match2:
                #print df.loc[[i]]
                #print df.loc[[j]]
                #row_dup.append(i)
                dup_file.write(str(i))
                dup_file.write("\n")
    dup_file.close()
    print "row numbers of duplicated matched truth jets written to file"

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
def drop_dup(df, file_name='default'):
    dup_file = open("dup/" + file_name + ".txt", "r")
    input_dup = dup_file.readlines()
    row_dup = []
    for i in range(0, len(input_dup)):
        row_dup.append((int)(input_dup[i][0:-1]))
    dup_file.close()
    df0 = df.drop(row_dup)
    print len(row_dup), "rows of duplicated matched truth jets removed from dataframe"
    return df0    

In [3]:
def fix_dphi(df, var='dphi'):
    b = (df.loc[df[var] < (-1) * np.pi]).index
    for i in range(0,len(b)):
        df.at[b[i], var] = 6.28318530718 + df.at[b[i], var]
    df_new = df.eval('abs_' + var + '= abs(' + var + ')')    
    print 'Fixed ' + var + ' delta phi range'
    return df_new

In [3]:
def count_njet(df, file_name='default'):
    njet_file = open("njet/" + file_name + ".txt", "w")
    max_njet = max(df['njets'])
    njet_array = np.zeros(max_njet)
    jet_index = 0
    while (jet_index < len(df)):
        event_index = np.asarray(df['event'])[jet_index]
        njet = 1
        end_index = jet_index + max_njet - 1
        if end_index >= len(df) - 1:
            end_index = len(df) - 1
        for j in range(jet_index + 1, end_index + 1):
            event_index2 = np.asarray(df['event'])[j]
            if event_index2 != event_index:
                break
            njet += 1
        njet_array[njet] += 1
        jet_index += njet
    for i in range(0, len(njet_array)):
        njet_file.write(str(njet_array[i]))
        njet_file.write('\n')
    njet_file.close()
    print "numbers of jet per event array is written to file"
    return njet_array

In [11]:
def read_njet(file_name="default"):
    njet_file = open("njet/" + file_name + ".txt", "r")
    input_njet = njet_file.readlines()
    njet_array = []
    for i in range(0, len(input_njet)):
        njet_array.append(int(float(input_njet[i][0:-1])))
    njet_file.close()
    return njet_array